<img style="border-radius: 0.5rem;" src="banner.jpg" />
<a style="margin-top: 1rem;" class="btn btn-lg btn-block btn-success" href="https://dsg.ucfsigai.org/fa18/signin/">
    Sign in (https://dsg.ucfsigai.org/fa18/signin/)
</a>

# Decision Trees
---
by: Chas Kane \([@ChasKane](github.com/ChasKane/)\), on 10 Oct 2018

In [10]:
import numpy as np
import pandas as pd


In [ ]:
df_raw = pd.read_csv("data/bulldozers/Train.csv")
# pd.set_option('display.max_columns', 500)
df_raw.head()